In [9]:
from agential.llm.llm import LLM
from dotenv import load_dotenv

load_dotenv()

import os

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))

In [10]:
from agential.cog.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_REACT
from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.expel.prompts import EXPEL_REFLEXION_REACT_INSTRUCTION_GSM8K, EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_GSM8K, GSM8K_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT, HOTPOTQA_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT, EXPEL_REFLEXION_REACT_INSTRUCTION_HOTPOTQA, EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA
from agential.cog.react.prompts import REACT_INSTRUCTION_GSM8K
from agential.cog.expel.memory import ExpeLExperienceMemory
from agential.cog.expel.strategies.math import ExpeLMathStrategy
from agential.cog.reflexion.agent import ReflexionReActAgent
import joblib


agent = ReflexionReActAgent(
    llm=llm, 
    benchmark="gsm8k",
)

exp = joblib.load("../../../tests/assets/expel/expel_experiences_10_fake.joblib")
strat = ExpeLMathStrategy(
    llm=llm, 
    reflexion_react_agent=agent,
    experience_memory=ExpeLExperienceMemory(experiences=exp)
)

question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = -9867630

out = strat.generate(
        question=question,
        key=key,
        examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
        prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_GSM8K,
        reflect_examples=GSM8K_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT,
        reflect_prompt=EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_GSM8K,
        reflect_strategy="reflexion",
        additional_keys={},
        reflect_additional_keys={},
        use_dynamic_examples = True,
        extract_insights = True,
        patience = 3,
        k_docs = 24,
        num_fewshots = 6,
        max_fewshot_tokens = 1500,
        reranker_strategy = None,
        reset=False
    )

<PROMPT COMPARE ======================================================================>
You are an advanced reasoning agent that can critique past task trajectories of youself. You will be given two previous task trials in which you were given access to a Docstore API environment and a question to answer: one successful and one unsuccessful trial. You failed the trial either because you guessed the wrong answer with Finish[<answer>], or you used up your set number of reasoning steps.

Here are the two previous trials to compare and critique:
TRIAL TASK:
The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? 

SUCCESSFUL TRIAL:
Thought: I need to focus on searching for the specific animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes, created by the same creator of "Wallace and Gromit."
Action: Search[animation comedy matched animat

In [ ]:
from agential.cog.expel.prompts import EXPEL_REFLEXION_REACT_INSTRUCTION_HUMANEVAL, EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_HUMANEVAL, HUMANEVAL_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT
from agential.cog.expel.strategies.code import ExpeLCodeStrategy
from agential.cog.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_REACT
from agential.cog.reflexion.prompts import HUMANEVAL_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_REACT_REFLECT_INSTRUCTION_HUMANEVAL


inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"


benchmark = "humaneval"

agent = ReflexionReActAgent(
    llm=llm, 
    benchmark="humaneval",
)

strat = ExpeLCodeStrategy(
    llm=llm, 
    reflexion_react_agent=agent,
)

out = strat.generate(
    question=question,
    key=key,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_HUMANEVAL,
    reflect_examples=HUMANEVAL_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT,
    reflect_prompt=EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_HUMANEVAL,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset_reflexion=True,
    reset=False,
)

In [ ]:
from agential.cog.expel.prompts import EXPEL_REFLEXION_REACT_INSTRUCTION_MBPP, EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP, MBPP_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT
from agential.cog.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_REACT
from agential.cog.reflexion.prompts import MBPP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP


question = "Write a python function to find the first repeated character in a given string."
key = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""


benchmark = "mbpp"

reflexion_react_agent = ReflexionReActAgent(
    llm=llm,
    benchmark=benchmark,
)
agent = ExpeLCodeStrategy(
    llm=llm,
    reflexion_react_agent=reflexion_react_agent,
)

out = agent.generate(
    question=question,
    key=key,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_MBPP,
    reflect_examples=MBPP_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT,
    reflect_prompt=EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP,
    reflect_strategy="reflexion",
    additional_keys={"tests": key},
    reflect_additional_keys={"tests": key},
    use_dynamic_examples=True,
    extract_insights=True,
    patience=3,
    k_docs=24,
    num_fewshots=6,
    max_fewshot_tokens=1500,
    reranker_strategy=None,
    reset_reflexion=True,
    reset=False,
)